# Extraer la informacion 

Se tiene como objetivo el crear el historico de los datos los cuales seran llevados al GitHub en formato .parquet

In [ ]:
# !pip install sodapy
# !pip install requests 
# !pip install pyarrow


In [1]:
import pandas as pd  
# Conexión a la API REST (SODA API)
import requests  
# Cliente oficial de Socrata 
from sodapy import Socrata  
# Manejo de archivos y sistema
import os  


In [2]:

dataset_id = "p6dx-8zbt"
url = f"https://www.datos.gov.co/resource/{dataset_id}.json"

# Contar el TOTAL de registros
params = {"$select": "count(*)"}
res = requests.get(url, params=params)
total_registros = int(res.json()[0]['count'])
print(f" TOTAL de registros en la base: {total_registros:,}")

# === 3. Consultar metadatos para saber cuántas variables tiene ===
meta_url = f"https://www.datos.gov.co/api/views/{dataset_id}.json"
meta_res = requests.get(meta_url).json()

# Extraer columnas declaradas en el metadata
vars_info = pd.DataFrame([{
    "Nombre": col['name'],
    "Campo (API)": col['fieldName'],
    "Tipo (API)": col['dataTypeName']
} for col in meta_res['columns']])

total_variables = len(vars_info)
print(f" TOTAL de variables en la base: {total_variables}")

# Vista previa de las variables
print("\n Variables disponibles:")
print(vars_info)



✅ TOTAL de registros en la base: 7,676,998
✅ TOTAL de variables en la base: 59

📌 Variables disponibles:
                                               Nombre  \
0                                             Entidad   
1                                         Nit Entidad   
2                                Departamento Entidad   
3                                      Ciudad Entidad   
4                                        OrdenEntidad   
5                                Entidad Centralizada   
6                                      ID del Proceso   
7                              Referencia del Proceso   
8                                                 PCI   
9                                   ID del Portafolio   
10                           Nombre del Procedimiento   
11                      Descripción del Procedimiento   
12                                               Fase   
13                   Fecha de Publicacion del Proceso   
14                        Fecha de Ultim

In [3]:
# ejemplo: 50,000 registros, ordenados por fecha
params = {
    "$limit": 50000,
    "$offset": 0,
    "$order": "fecha_de_publicacion_del ASC"   # menor a mayor
}
res = requests.get(url, params=params)
df = pd.DataFrame(res.json())

# Convertir SOLO la variable "Fecha de Publicacion del Proceso"
fecha_col = "fecha_de_publicacion_del"
if fecha_col in df.columns:
    df[fecha_col] = pd.to_datetime(df[fecha_col], errors="coerce")

print(f"\n📊 Primer lote descargado")
print(f"   - Registros en el lote: {len(df):,}")
print(f"   - Variables en el lote: {df.shape[1]}")

print("\n Primeros 5 registros (con fecha ya convertida):")
print(df.head())


📊 Primer lote descargado
   - Registros en el lote: 50,000
   - Variables en el lote: 57

📌 Primeros 5 registros (con fecha ya convertida):
                                             entidad nit_entidad  \
0  EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...   899999094   
1  EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...   899999094   
2  EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...   899999094   
3  EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...   899999094   
4  EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...   899999094   

         departamento_entidad ciudad_entidad ordenentidad       codigo_pci  \
0  Distrito Capital de Bogotá         Bogotá  Territorial  Descentralizada   
1  Distrito Capital de Bogotá         Bogotá  Territorial  Descentralizada   
2  Distrito Capital de Bogotá         Bogotá  Territorial  Descentralizada   
3  Distrito Capital de Bogotá         Bogotá  Territorial  Descentralizada   
4  Distrito Capital de Bogotá         Bogotá  Territorial  D

In [4]:

params = {
    "$limit": 100000,
    "$offset": 0,
    "$order": "fecha_de_publicacion_del ASC"
}
print("\nEjemplos de valores en bruto:")
print(df["fecha_de_publicacion_del"].dropna().head(10).tolist())

# Convertir la fecha con formato flexible
df["fecha_de_publicacion_del"] = pd.to_datetime(
    df["fecha_de_publicacion_del"],
    errors="coerce",
    utc=True,               # mantener en UTC si viene con hora
)

# Validar  NA
total_na = df["fecha_de_publicacion_del"].isna().sum()
print(f"\n Total fechas nulas después de conversión: {total_na:,}")

# Guardar en .parquet
ruta = r"C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes\lote_1_prueba.parquet"
df.to_parquet(ruta, engine="pyarrow", index=False)
print(f" Lote guardado en: {ruta}")

# para validar
df_parquet = pd.read_parquet(ruta)
print("\n Primeros 5 registros con fecha convertida:")
print(df_parquet[["fecha_de_publicacion_del"]].head())



Ejemplos de valores en bruto:
[Timestamp('2015-04-16 00:00:00'), Timestamp('2015-04-20 00:00:00'), Timestamp('2015-04-20 00:00:00'), Timestamp('2015-04-20 00:00:00'), Timestamp('2015-04-20 00:00:00'), Timestamp('2015-04-20 00:00:00'), Timestamp('2015-04-20 00:00:00'), Timestamp('2015-04-20 00:00:00'), Timestamp('2015-04-20 00:00:00'), Timestamp('2015-04-20 00:00:00')]

📊 Total fechas nulas después de conversión: 0
✅ Lote guardado en: C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes\lote_1_prueba.parquet

Primeros 5 registros con fecha convertida:
   fecha_de_publicacion_del
0 2015-04-16 00:00:00+00:00
1 2015-04-20 00:00:00+00:00
2 2015-04-20 00:00:00+00:00
3 2015-04-20 00:00:00+00:00
4 2015-04-20 00:00:00+00:00


In [5]:
from datetime import datetime, timedelta
ruta_salida = r"C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes"

# fecha mínima y máxima
query = {
    "$select": "min(fecha_de_publicacion_del) as min_fecha, max(fecha_de_publicacion_del) as max_fecha"
}
res = requests.get(url, params=query)
fechas = res.json()[0]

min_fecha = pd.to_datetime(fechas["min_fecha"])
max_fecha = pd.to_datetime(fechas["max_fecha"])

print(f" Rango total: {min_fecha.date()} → {max_fecha.date()}")

# Generar semestres
semestres = []
anio = min_fecha.year
while anio <= max_fecha.year:
    # Primer semestre
    inicio_s1 = datetime(anio, 1, 1)
    fin_s1 = datetime(anio, 6, 30)

    # Segundo semestre
    inicio_s2 = datetime(anio, 7, 1)
    fin_s2 = datetime(anio, 12, 31)

    semestres.append((anio, "S1", inicio_s1, fin_s1))
    semestres.append((anio, "S2", inicio_s2, fin_s2))

    anio += 1

# Ajustar primer y último semestre según rango real
semestres = [
    (a, s, max(inicio, min_fecha), min(fin, max_fecha))
    for a, s, inicio, fin in semestres
    if fin >= min_fecha and inicio <= max_fecha
]

# Descargar y guardar cada semestre
for anio, semestre, inicio, fin in semestres:
    print(f"\n📥 Descargando {anio}_{semestre} ({inicio.date()} → {fin.date()})...")

    params = {
        "$where": f"fecha_de_publicacion_del between '{inicio.date()}' and '{fin.date()}'",
        "$limit": 100000,  # Socrata sigue limitando, toca paginar
        "$offset": 0,
        "$order": "fecha_de_publicacion_del ASC"
    }

    df_total = pd.DataFrame()

    while True:
        res = requests.get(url, params=params)
        datos = res.json()
        if not datos:
            break

        df = pd.DataFrame(datos)

        # === Arreglo de la fecha ===
        if "fecha_de_publicacion_del" in df.columns:
            df["fecha_de_publicacion_del"] = pd.to_datetime(
                df["fecha_de_publicacion_del"], errors="coerce", utc=True
            )

        df_total = pd.concat([df_total, df], ignore_index=True)

        # Avanzar en la paginación
        params["$offset"] += params["$limit"]

    # === Guardar parquet ===
    archivo_out = os.path.join(ruta_salida, f"lote_{anio}_{semestre}.parquet")
    df_total.to_parquet(archivo_out, engine="pyarrow", index=False)
    print(f"   ✅ Guardado {len(df_total):,} registros en {archivo_out}")

print("\n🚀 Descarga completa y archivada por semestres")


📅 Rango total: 2015-04-16 → 2025-09-08

📥 Descargando 2015_S1 (2015-04-16 → 2015-06-30)...
   ✅ Guardado 313 registros en C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes\lote_2015_S1.parquet

📥 Descargando 2015_S2 (2015-07-01 → 2015-12-31)...
   ✅ Guardado 2,959 registros en C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes\lote_2015_S2.parquet

📥 Descargando 2016_S1 (2016-01-01 → 2016-06-30)...
   ✅ Guardado 3,598 registros en C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes\lote_2016_S1.parquet

📥 Descargando 2016_S2 (2016-07-01 → 2016-12-31)...
   ✅ Guardado 6,463 registros en C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes\lote_2016_S2.parquet

📥 Descargando 2017_S1 (2017-01-01 → 2017-06-30)...
   ✅ Guardado 15,752 registros en C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes\lote_2017_S1.parquet

📥 Descargando 20

In [6]:
import shutil
origen = "HISTORICO_SECOP.ipynb"
destino = r"C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes\HISTORICO_SECOP.ipynb"

shutil.copy(origen, destino)
print(f"✅ Notebook copiado a: {destino}")


✅ Notebook copiado a: C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes\HISTORICO_SECOP.ipynb
